In [ ]:
import onnxruntime as ort
import glob as gb
import pandas as pd
import numpy as np
import math

# ==============================================================
# Helper: run ONNX model
# ==============================================================
def onnx_predict(session, X):
    """
    Run inference with an ONNX model.

    Parameters
    ----------
    session : onnxruntime.InferenceSession
    X : np.ndarray, shape (N, 2)
        Physical inputs (NOT standardized), float32/float64

    Returns
    -------
    np.ndarray, shape (N, n_outputs)
        Model predictions
    """
    input_name = session.get_inputs()[0].name
    out_names = [o.name for o in session.get_outputs()]
    preds = session.run(out_names, {input_name: X.astype(np.float32)})

    # Most models expose a single output tensor -> preds[0]
    return preds[0]


# ==============================================================
# Read FlameMaster data
# ==============================================================
fname = './../neural_network/data/chemtable_FVV_2D_Enthalpy/*.kg'
files = gb.glob(fname)
nfiles = len(files)

# Read column names from the second line of the first file
with open(files[0], 'r') as f:
    lines = f.readlines()
    column_names = lines[1].strip().split('\t')

data_flameMaster = []
for f in files:
    df_temp = pd.DataFrame(
        np.loadtxt(f, skiprows=2, dtype=np.float64),
        columns=column_names
    )
    data_flameMaster.append(df_temp)

df_flameMaster_all = pd.concat(data_flameMaster, ignore_index=True)

# Compute diffusivity
df_flameMaster_all['Diff [kg/ms]'] = (
    df_flameMaster_all['lambda [W/mK]'] / df_flameMaster_all['cp [J/kgK]']
)

# Inputs / outputs
input_data = ['ProgVar', 'TotalEnthalpy [J/kg]']
output_data = [
    'ProdRateProgVar [kg/m^3s]',
    'temperature [K]',
    'Y-CO',
    'density',
    'mu [kg/ms]',
    'cp [J/kgK]',
    'Diff [kg/ms]'
]

# Shift enthalpy relative to a reference value
referenceEnthalpy = 276240
df_flameMaster_all['TotalEnthalpy [J/kg]'] = (
    df_flameMaster_all['TotalEnthalpy [J/kg]'] - referenceEnthalpy
)

X_all = df_flameMaster_all[input_data].to_numpy()
Z_all = df_flameMaster_all[output_data].to_numpy()

# Train/test mask from the original NN training
mask = np.load("./../neural_network/train_test_mask.npy")

X_train = X_all[mask].astype(np.float32)
Z_train = Z_all[mask].astype(np.float32)
X_test  = X_all[~mask].astype(np.float32)
Z_test  = Z_all[~mask].astype(np.float32)

# Training statistics (useful to define nominal point)
x_mean = X_train.mean(axis=0).astype(np.float64)
x_std  = X_train.std(axis=0).astype(np.float64) + 1e-12


# ==============================================================
# ONNX model session
# ==============================================================
onnx_path = "./../neural_network/saved_models/NN_model_for_uq_analysis.onnx"
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])


# ==============================================================
# Hermite basis (for xi ~ N(0,1))
# IMPORTANT:
#   Hermite PCE assumes the random variables (xi) are standard normal.
#   You should build the PCE in xi-space, NOT by "standardizing noisy X".
# ==============================================================
def hermite_polynomial(n, x):
    # Probabilists' Hermite recursion (consistent with Gaussian-weighted orthogonality)
    if n == 0:
        return np.ones_like(x)
    elif n == 1:
        return x
    return x * hermite_polynomial(n - 1, x) - (n - 1) * hermite_polynomial(n - 2, x)

def orthonormal_hermite(n, x):
    # Orthonormal version under standard normal measure
    return hermite_polynomial(n, x) / np.sqrt(math.factorial(n))

def total_order_basis_2d(p):
    # All (i, j) such that i + j <= p
    return [(i, j) for i in range(p + 1) for j in range(p + 1) if i + j <= p]

def design_matrix_2d_hermite(XI, basis):
    """
    Build design matrix A for 2D Hermite basis evaluated at xi.

    Parameters
    ----------
    XI : np.ndarray, shape (N, 2)
        Standard normal samples (xi1, xi2) ~ N(0,1)
    basis : list of tuple(int, int)
        Multi-index basis (i, j)

    Returns
    -------
    A : np.ndarray, shape (N, P)
        Design matrix where A[n,k] = H_i(xi1[n]) * H_j(xi2[n])
        using ORTHONORMAL Hermite polynomials
    """
    xi1 = XI[:, 0]
    xi2 = XI[:, 1]

    N = XI.shape[0]
    P = len(basis)
    A = np.zeros((N, P), dtype=np.float64)

    max_deg = max(max(i, j) for i, j in basis)
    H1 = np.stack([orthonormal_hermite(n, xi1) for n in range(max_deg + 1)], axis=1)
    H2 = np.stack([orthonormal_hermite(n, xi2) for n in range(max_deg + 1)], axis=1)

    for k, (i, j) in enumerate(basis):
        A[:, k] = H1[:, i] * H2[:, j]
    return A


# ==============================================================
# PCE training: sample in xi-space, map to physical X, evaluate ONNX
# ==============================================================
order = 12         # Start low (3–5). Increase only if stable.
sigma_rel = 0.05   # 5% relative uncertainty around a nominal point
N_fit = 1820       # Enough for 2D at low order; can increase if needed

rng = np.random.default_rng(0)

# 1) Draw xi samples from standard normal
XI_fit = rng.standard_normal(size=(N_fit, 2))

# 2) Choose a nominal point in physical space (here: training mean)
x0 = x_mean.copy()

# 3) Define physical-space standard deviations
#    Using relative uncertainty: sigma_phys = sigma_rel * |x0|
#    Add eps to avoid sigma=0 if x0 is near zero.
eps = 1e-8
sigma_phys = sigma_rel * np.maximum(np.abs(x0), eps)

# 4) Map xi -> X (physical)
X_fit_phys = x0 + XI_fit * sigma_phys

# 5) Clip to training-domain bounds to stay within interpolation regime
c_min, c_max = X_train[:, 0].min(), X_train[:, 0].max()
h_min, h_max = X_train[:, 1].min(), X_train[:, 1].max()

X_fit_phys[:, 0] = np.clip(X_fit_phys[:, 0], c_min, c_max)
X_fit_phys[:, 1] = np.clip(X_fit_phys[:, 1], h_min, h_max)

# 6) Evaluate the ONNX model at these physical inputs
Z_fit_phys = onnx_predict(sess, X_fit_phys)
Z_fit_phys = np.asarray(Z_fit_phys, dtype=np.float64)  # shape (N_fit, n_outputs)


# ==============================================================
# Fit PCE coefficients
# ==============================================================
basis = total_order_basis_2d(order)
print("order:", order, "| P terms:", len(basis))

A_fit = design_matrix_2d_hermite(XI_fit, basis)
print("A_fit shape:", A_fit.shape, "| Z_fit_phys shape:", Z_fit_phys.shape)

# Least squares fit for multi-output
coeffs, *_ = np.linalg.lstsq(A_fit, Z_fit_phys, rcond=None)  # shape (P, n_outputs)

# Optional: ridge stabilization if you still see instability
# lam = 1e-6
# AtA = A_fit.T @ A_fit
# AtZ = A_fit.T @ Z_fit_phys
# coeffs = np.linalg.solve(AtA + lam*np.eye(AtA.shape[0]), AtZ)


# ==============================================================
# Mean and variance from orthonormal PCE coefficients
# For orthonormal basis under N(0,1):
#   mean = c0
#   var  = sum_{k>0} c_k^2
# ==============================================================
mean_pce = coeffs[0, :]                      # (n_outputs,)
var_pce  = np.sum(coeffs[1:, :]**2, axis=0)  # (n_outputs,)
std_pce  = np.sqrt(var_pce)

df_uq = pd.DataFrame({
    "Output": output_data,
    "Mean": mean_pce,
    "Variance": var_pce,
    "Std": std_pce
})

df_uq


# ==============================================================
# Extra safety check (optional):
# Compare coefficient-based variance vs sample-based variance of the PCE surrogate
# ==============================================================
# Y_hat = A_fit @ coeffs
# mean_mc_pce = Y_hat.mean(axis=0)
# var_mc_pce  = Y_hat.var(axis=0, ddof=1)

# df_check = pd.DataFrame({
#     "Output": output_data,
#     "Mean_coeff": mean_pce,
#     "Mean_mc": mean_mc_pce,
#     "Var_coeff": var_pce,
#     "Var_mc": var_mc_pce,
# })

# print("\nSanity check (coeff-based vs sample-based):")
# df_check


order: 12 | P terms: 91
A_fit shape: (1820, 91) | Z_fit_phys shape: (1820, 7)

Sanity check (coeff-based vs sample-based):


,Output,Mean_coeff,Mean_mc,Var_coeff,Var_mc
0,ProdRateProgVar [kg/m^3s],349.208756,344.533970,3.577239e+04,3.514543e+04
1,temperature [K],1464.723516,1463.545848,1.602384e+03,1.602939e+03
2,Y-CO,0.010566,0.010556,1.406820e-07,1.422546e-07
3,density,0.931452,0.932206,6.602448e-04,6.647742e-04
4,mu [kg/ms],0.000055,0.000055,2.629792e-11,1.079257e-12
5,cp [J/kgK],1321.378143,1321.225728,2.684026e+01,2.700359e+01
6,Diff [kg/ms],0.000070,0.000070,3.514459e-11,1.871865e-12
